# 02 - Clusterização dos Fornecedores

Neste notebook, aplico técnicas de clusterização para identificar padrões de comportamento entre os fornecedores contratados pelo IFSP no período de 2014 a 2024. A análise é feita com base na frequência de atuação, no valor total empenhado e na quantidade de registros por CNPJ.

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

## Leitura do arquivo consolidado
Agora carrego o dataset unificado gerado na etapa de pré-processamento.

In [ ]:
df = pd.read_csv('../dados/OBRAS_IFSP_2014_2024.csv')
df.head()

## Agrupamento por CNPJ
Agrupo os dados por CNPJ para extrair variáveis que servirão como base para o modelo de clusterização.

In [ ]:
# Agrupando por CNPJ
df_cnpjs = df.groupby('cnpj').agg({
    'valor_empenhado': 'sum',
    'valor_pago': 'sum',
    'n_empenho': 'count',
    'ano': pd.Series.nunique
}).reset_index()

# Renomeando colunas
df_cnpjs.columns = ['cnpj', 'total_empenhado', 'total_pago', 'num_empenhos', 'anos_atuacao']

## Padronização dos dados
Os dados são escalonados para evitar que variáveis com valores maiores dominem o processo de agrupamento.

In [ ]:
# Padronizando os dados para o KMeans
scaler = StandardScaler()
dados_cluster = scaler.fit_transform(df_cnpjs[['total_empenhado', 'num_empenhos', 'anos_atuacao']])

## Aplicação do KMeans
Aplico o algoritmo com 3 clusters, conforme estratégia exploratória da análise.

In [ ]:
# Criando modelo de clusterização
kmeans = KMeans(n_clusters=3, random_state=42)
df_cnpjs['cluster'] = kmeans.fit_predict(dados_cluster)

## Visualização dos clusters
Crio um gráfico de dispersão simples para ilustrar a separação dos grupos formados pelo modelo.

In [ ]:
# Gráfico de dispersão por cluster
plt.figure(figsize=(10,6))
plt.scatter(df_cnpjs['total_empenhado'], df_cnpjs['num_empenhos'], c=df_cnpjs['cluster'], cmap='viridis')
plt.xlabel('Total Empenhado')
plt.ylabel('Número de Empenhos')
plt.title('Distribuição dos CNPJs por Cluster')
plt.grid(True)
plt.show()

## Exportação dos dados com cluster
Salvo os resultados para uso posterior em visualizações e análises qualitativas.

In [ ]:
# Exportando o resultado com os clusters identificados
df_cnpjs.to_csv('../output/fornecedores_com_clusters.csv', index=False)